# Clase - Computación Distribuida

## Pyspark Streaming Hands-on 
#### Marcelo Medel Vergara - Diplomado Data Engineer USACH


### Input Sources

Structured Streaming nos permite recibir datos desde distintas fuentes de datos:

- Apache Kafka 0.10 - https://kafka.apache.org/ 
- Archivos en un sistema de archivos distribuidos 
    - HDFS - https://hadoop.apache.org/docs/r1.2.1/hdfs_design.html 
    - Amazon - S3 https://aws.amazon.com/es/s3/ 
- Socket local para propósitos de testing - https://docs.python.org/3/library/socket.html 

### Sinks 

Un sink determina dónde y cómo se almacenan los resultados del procesamiento de streaming

- **File Sink**: útil para escribir los resultados del procesamiento en archivos (CSV, Parquet, JSON).

- **Console Sink**: Los resultados del procesamiento se imprimen en la consola de salida de PySpark.

- **Kafka Sink**: Se puede enviar los resultados del procesamiento a Kafka.

- **JDBC Sink**: Puede escribir los datos en una tabla de una base de datos compatible con JDBC.



In [1]:
import findspark
findspark.find()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("streaming").getOrCreate()

spark.active()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/16 20:40:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.json("data/streaming/streaming.json")
data.printSchema()
data.show()

root
 |-- conn_country: string (nullable = true)
 |-- episode_name: string (nullable = true)
 |-- episode_show_name: string (nullable = true)
 |-- incognito_mode: boolean (nullable = true)
 |-- ip_addr_decrypted: string (nullable = true)
 |-- master_metadata_album_album_name: string (nullable = true)
 |-- master_metadata_album_artist_name: string (nullable = true)
 |-- master_metadata_track_name: string (nullable = true)
 |-- ms_played: long (nullable = true)
 |-- offline: boolean (nullable = true)
 |-- offline_timestamp: long (nullable = true)
 |-- platform: string (nullable = true)
 |-- reason_end: string (nullable = true)
 |-- reason_start: string (nullable = true)
 |-- shuffle: boolean (nullable = true)
 |-- skipped: string (nullable = true)
 |-- spotify_episode_uri: string (nullable = true)
 |-- spotify_track_uri: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- user_agent_decrypted: string (nullable = true)
 |-- username: string (nullable = true)

+------------+---

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, BooleanType, TimestampType

schema = StructType([
    StructField("conn_country", StringType(), True),
    StructField("episode_name", StringType(), True),
    StructField("episode_show_name", StringType(), True),
    StructField("incognito_mode", BooleanType(), True),
    StructField("ip_addr_decrypted", StringType(), True),
    StructField("master_metadata_album_album_name", StringType(), True),
    StructField("master_metadata_album_artist_name", StringType(), True),
    StructField("master_metadata_track_name", StringType(), True),
    StructField("ms_played", LongType(), True),
    StructField("offline", BooleanType(), True),
    StructField("offline_timestamp", LongType(), True),
    StructField("platform", StringType(), True),
    StructField("reason_end", StringType(), True),
    StructField("reason_start", StringType(), True),
    StructField("shuffle", BooleanType(), True),
    StructField("skipped", BooleanType(), True),
    StructField("spotify_episode_uri", StringType(), True),
    StructField("spotify_track_uri", StringType(), True),
    StructField("ts", TimestampType(), True),
    StructField("user_agent_decrypted", StringType(), True),
    StructField("username", StringType(), True)
])

data_stream = spark.readStream.schema(schema).json("data/streaming/")
data_stream = spark.readStream.json("data/streaming/")
data_stream.printSchema()

IllegalArgumentException: Schema must be specified when creating a streaming source DataFrame. If some files already exist in the directory, then depending on the file format you may be able to create a static DataFrame on that directory with 'spark.read.load(directory)' and infer schema from it.

### Output modes

- **Complete mode**:
    - Envía todo el resultado calculado al destino.
    - Útil para datos de estado que cambian con el tiempo.
    - Útil cuando el destino no admite actualizaciones a nivel de fila.
- **Update mode**:
    - Envía solo las filas que difieren de la última escritura al destino.
    - Destino debe admitir actualizaciones a nivel de fila.
    - Si la consulta no contiene agregaciones, es equivalente al modo de *append*
- **Append mode**:
    - Nuevas filas se envían al destino especificado
    - Garantiza que cada fila se envíe una vez y solo una vez
    - Destino debe ser tolerante a fallos

### Triggers

Los triggers en Spark Structured Streaming controlan cuándo se envía la data al destino. Por defecto, el streaming comienza a procesar datos tan pronto como el trigger anterior termina. Los triggers son útiles para evitar sobrecargar el destino con demasiadas actualizaciones o para controlar el tamaño de los archivos de salida. Existen dos tipos de triggers:

- **Processing Time Trigger**: Se especifica una duración (ej: “10 segundos”) y Spark esperará múltiplos de esa duración para enviar los datos. Si el procesamiento no termina antes del siguiente *trigger*, Spark esperará al siguiente punto en lugar de disparar inmediatamente.
- **Once Trigger**: Permite ejecutar un trabajo de streaming solo una vez. Es útil tanto en desarrollo (para probar aplicaciones con un solo conjunto de datos) como en producción (para ejecutar trabajos manualmente a una baja frecuencia, ahorrando recursos).


## Streaming from socket

Usar este código para simular envío de mensajes a través de un socket --> https://github.com/MarceloMedel/SPARK-DE-USACH/blob/main/streaming_file.ipynb


In [27]:
import findspark
findspark.find()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("streaming-socket").getOrCreate()

spark.active()